In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np
import pandas_profiling
import pickle
from IPython.display import Image 

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.4f}".format)
pd.set_option("display.max_info_rows", 200)


import surprise
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD, SVDpp, BaselineOnly, NMF, CoClustering
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")
sns.set_context("paper", font_scale=1.8)
sns.set_palette("hls", 15)

In [89]:
import nltk
from nltk.corpus import gutenberg, stopwords
from nltk.collocations import *
from nltk import FreqDist
from nltk import word_tokenize
from gensim.models import Word2Vec
import string

# Dataset

In [2]:
# List of all products from Sephora as of June 24 with binary columns for categories

products = pd.read_csv("df_sephora.csv")

In [4]:
products.info(null_counts=True, max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9007 entries, 0 to 9006
Data columns (total 192 columns):
id                                   9007 non-null object
name                                 9007 non-null object
brand_id                             9007 non-null int64
brand_name                           9007 non-null object
short_desc                           8997 non-null object
long_desc                            8997 non-null object
item                                 9007 non-null int64
item_name                            7696 non-null object
list_price                           9007 non-null float64
variation                            9007 non-null object
variants                             3930 non-null object
price_low                            9007 non-null float64
price_high                           9007 non-null float64
is_limited_edition                   9007 non-null int64
rating                               8792 non-null float64
review_count         

In [3]:
df = pd.read_csv("user_item_rating_raw.csv")
df = df[["uid", "author_id", "product_id", "rating"]].copy()

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
author_ids = list(df.uid.unique())
product_ids = list(df.product_id.unique())

In [349]:
# pickle.dump(author_ids, open("uid.pkl", "wb"))

In [5]:
del df

In [6]:
selected_products = products[products.id.isin(product_ids)]
selected_products.shape

(6839, 192)

In [41]:
# selected_products.to_csv("df_sephora_selected.csv", index=False)

In [296]:
keyword = ["activity"]

keyword = "|".join(
    ["\s" + key + "\s|\s" + key + "\,|\s" + key + "\." for key in keyword])

selected_products[selected_products.long_desc.str.contains(keyword,
                                                           flags=re.IGNORECASE,
                                                           regex=True,
                                                           na=False)]

,id,name,brand_id,brand_name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_long,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men
353,P408678,Midnight Secret Late Night Recovery Treatment Anti-Fatigue Serum,1132,Guerlain,"A lightweight liquid, nighttime treatment that reduces the appearance of puffiness and while smoothing away the visible signs of tiredness.","What it is: A lightweight liquid, nighttime treatment that reduces the appearance of puffiness and while smoothing away the visible signs of tiredness. Skin Type: Normal, Dry, Combination, and Oily Skincare Concerns: Puffiness, Loss of Firmness and Elasticity, and Dark Circles Formulation: Lightweight Cream Highlighted Ingredients: - Hydronoctine Complex: A unique blend of eight ingredients glycyrrhetinic acid, oryzanol, tocopheryl linoleate, tocopheryl acetate, sodium lactate, sodium hyaluronate, ginkgo biloba, and glycoproteins that work together to reduce the signs of fatigue. - Gingko Biloba Extract: A plant well known for its benefits on microcirculation and detoxification. - Glycoproteins: Support natural cellular activity, helping to enhance overall skin tone. What Else You Need to Know: Wake up looking rested and revitalized. Guerlain's secret to looking rested after a long night is formulated with eight effective ingredients that help reduce the appearance of puffiness and smooth away visible signs of tiredness. Clinical Results: In clinical tests: - 79% noted t

In [293]:
selected_products.long_desc = selected_products.long_desc.str.replace("active ingredient", "???", case=False)
selected_products.long_desc = selected_products.long_desc.str.replace("active molecules", "??? molecules", case=False)
selected_products.long_desc = selected_products.long_desc.str.replace("active blemish", "??? blemish", case=False)
selected_products.long_desc = selected_products.long_desc.str.replace("active condition", "??? condition", case=False)

In [ ]:
selected_products.long_desc.str.replace("active")

In [203]:
long_desc = " ".join(fragrance[fragrance.long_desc.isna()==False].long_desc.values)

In [204]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"

In [205]:
tokens_raw = [word.lower() for word in nltk.regexp_tokenize(long_desc, pattern)]

In [206]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)
stopwords_list += ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
stopwords_list += [
    "family", "scent", "type", "fragrance", "key", "notes", "description",
    "characterized", "pattern", "inspired", "literally", "describe", "used",
    'bottle', "de", "eau", "free", "sephora", "ingredients", "parfum", "oz",
    "ml", "know", "like", "toilette", "woman", "formulated", "top"
]

In [207]:
tokens_stopped = [word for word in tokens_raw if word not in stopwords_list]
freqdist = FreqDist(tokens_stopped)
freqdist.most_common(200)

[('floral', 708),
 ('fresh', 605),
 ('warm', 520),
 ('rose', 412),
 ('florals', 382),
 ('jasmine', 369),
 ('clean', 350),
 ('vanilla', 346),
 ('white', 297),
 ('musk', 288),
 ('amber', 265),
 ('citrus', 258),
 ('bergamot', 249),
 ('woody', 244),
 ('patchouli', 243),
 ('orange', 229),
 ('sensual', 227),
 ('sweet', 227),
 ('new', 221),
 ('sandalwood', 217),
 ('accord', 216),
 ('heart', 213),
 ('perfume', 209),
 ('classic', 203),
 ('list', 200),
 ('need', 198),
 ('blend', 195),
 ('flower', 194),
 ('woods', 194),
 ('spicy', 192),
 ('fragrances', 190),
 ('else', 182),
 ('black', 181),
 ('style', 177),
 ('essence', 176),
 ('skin', 165),
 ('blossom', 164),
 ('fruity', 164),
 ('modern', 161),
 ('love', 159),
 ('full', 157),
 ('unique', 155),
 ('oil', 151),
 ('wood', 149),
 ('without', 148),
 ('scents', 147),
 ('pink', 142),
 ('earthy', 142),
 ('vetiver', 142),
 ('cedarwood', 140),
 ('made', 136),
 ('create', 135),
 ('light', 134),
 ('phthalates', 132),
 ('green', 131),
 ('signature', 130),
 ('

# Model 3

In [299]:
rel = str(input("Relationship to recipient: ")).lower()

gender = str(input("He, She or They:  (enter 0 if unknown)")).lower()

budget_max = float(input("Maximum budget in US$: "))
budget_min = float(input("Minimum in US$: "))

age = int(input("Age: (please provide estimate if unknown): "))

keywords = str(
        input("3 words to decribe recipient:  Example: young, fresh, traveler (separate using comma ',')")).replace(
            " ", "").replace("-", "").lower().split(",")

Relationship to recipient: brother
He, She or They:  (enter 0 if unknown)he
Maximum budget in US$: 500
Minimum in US$: 150
Age: (please provide estimate if unknown): 40
3 words to decribe recipient:  Example: young, fresh, traveler (separate using comma ',')active, fun, old


In [306]:
[i for i in author_ids if i.startswith("35to44")]

['35to44_0_0_0_0_0_0_0',
 '35to44_0_0_0_0_blonde_0_0',
 '35to44_0_0_0_brown_brunette_0_0',
 '35to44_0_0_0_hazel_0_0_0',
 '35to44_0_combination_0_0_0_0_0',
 '35to44_0_combination_blackheads_0_0_0_0',
 '35to44_0_combination_sensitivity_green_0_0_0',
 '35to44_0_combination_sunDamage_0_0_0_0',
 '35to44_0_oily_acne_brown_auburn_0_0',
 '35to44_dark_combination_0_brown_auburn_0_0',
 '35to44_dark_combination_0_brown_auburn_0_CurlyEnhancing',
 '35to44_dark_combination_0_brown_black_0_0',
 '35to44_dark_combination_0_brown_black_0_Damage',
 '35to44_dark_combination_0_brown_black_0_Frizz',
 '35to44_dark_combination_0_brown_black_0_StraighteningSmoothing',
 '35to44_dark_combination_0_brown_black_0_Thinning',
 '35to44_dark_combination_0_brown_blonde_0_0',
 '35to44_dark_combination_0_brown_brunette_0_0',
 '35to44_dark_combination_0_brown_brunette_0_Dandruff',
 '35to44_dark_combination_0_brown_brunette_0_StraighteningSmoothing',
 '35to44_dark_combination_0_brown_gray_0_0',
 '35to44_dark_combination_0_

In [326]:
products_to_consider = get_products2(["young", "fresh", "traveler"], "brother", "he")

In [333]:
products_to_consider = []

In [338]:
men = pickle.load(open("for_men.pkl", "rb"))

keywords = ["young", "fresh", "traveler"]
gender = "he"

keyword_string = "|".join(
    ["\s" + key + "\s|\s" + key + "\,|\s" + key + "\." for key in keywords])

if gender == "he":
    products_to_consider = list(
        selected_products[(selected_products.long_desc.str.contains(
            keyword_string, flags=re.IGNORECASE, regex=True, na=False))
                          & (selected_products.id.isin(men))].id.unique())
#     for i in products_to_consider:
#         if i not in men:
#             products_to_consider.remove(i)
#         else:
#             continue

In [354]:
uids = pickle.load(open("uid.pkl", "rb"))
similar_uids = [uid for uid in uids if uid.startswith("35to44") and not uid.endswith("_0_0_0_0_0_0_0")]
similar_uids.sort()


['35to44_0_0_0_0_blonde_0_0',
 '35to44_0_0_0_brown_brunette_0_0',
 '35to44_0_0_0_hazel_0_0_0',
 '35to44_0_combination_0_0_0_0_0',
 '35to44_0_combination_blackheads_0_0_0_0',
 '35to44_0_combination_sensitivity_green_0_0_0',
 '35to44_0_combination_sunDamage_0_0_0_0',
 '35to44_0_oily_acne_brown_auburn_0_0',
 '35to44_dark_combination_0_brown_auburn_0_0',
 '35to44_dark_combination_0_brown_auburn_0_CurlyEnhancing',
 '35to44_dark_combination_0_brown_black_0_0',
 '35to44_dark_combination_0_brown_black_0_Damage',
 '35to44_dark_combination_0_brown_black_0_Frizz',
 '35to44_dark_combination_0_brown_black_0_StraighteningSmoothing',
 '35to44_dark_combination_0_brown_black_0_Thinning',
 '35to44_dark_combination_0_brown_blonde_0_0',
 '35to44_dark_combination_0_brown_brunette_0_0',
 '35to44_dark_combination_0_brown_brunette_0_Dandruff',
 '35to44_dark_combination_0_brown_brunette_0_StraighteningSmoothing',
 '35to44_dark_combination_0_brown_gray_0_0',
 '35to44_dark_combination_0_brown_red_0_0',
 '35to44_

In [339]:
selected_products[selected_products.id.isin(products_to_consider)].for_men.value_counts()

1    73
Name: for_men, dtype: int64

In [340]:
# products = pd.read_csv("df_sephora_selected.csv")
topn = get_topn("35to44_0_0_0_0_0_0_0", products_to_consider, n=len(selected_products))
topn = topn.merge(products, how="left", left_on="iid", right_on="id")
topn

,iid,est,id,name,brand_id,brand_name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_long,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men
0,P180727,4.9164,P180727,L'Homme,1070,Yves Saint Laurent,"Infused with the force of attraction of a man with style and sensuality, this fresh, woody fragrance is effervescent with bright, sparkling notes of bergamot, ginger, and vetiver.","Infused with the force of attraction of a man with style and sensuality, this fresh, woody fragrance is effervescent with bright, sparkling notes of bergamot, ginger, and vetiver. A fragrance of contrasts it is a unique combination of luxury, art, and modernity that exudes a timeless elegance. Notes: Bergamot, Cedar, Ozonic Notes, White Ginger, Basil Flower, White Pepper, Violet Leaves, Cedarwood, Haiti Vetiver, Tonka Bean. Style: Magnetic. Sensual. Intriguing.",987917,2 oz/ 60 mL Eau de Toilette Spray,74.0000,Size + Concentration + Formulation,"[{1: {'item': '1025824', 'item_name': '1.3 oz Eau de Toilette Spray', 'list_price': 60.0}}, {2: {'item': '987917', 'item_name': '2 oz/ 60 mL Eau de Toilette Spray', 'list_price': 74.0}}, {3: {'item': '987925', 'item_name': '3.3 oz/ 100 mL Eau de Toilette Spray', 'list_price': 95.0}}, {4: {'item': '1668896', 'item_name': '6.7 oz/ 200 mL Eau de Toilette Spray', 'list_price': 135.0}}]",60.0000,135.0000,0,4.8471,870,16719,"{'cat60150': 'Cologne', 'cat1230040': 'Men', 'cat160006': 'Fragrance'}",https://www.sephora.com/pr

In [344]:
topn[(topn.price_high <= 500)
                            & (topn.price_low >= 150)].reset_index(
                                drop=True).shape

(4, 194)

In [302]:
top_ = get_reco2(age=age,
     keywords = keywords,
     rel = rel,
     gender = gender,
     budget_max = budget_max,
     budget_min = budget_min,
     n=15)

top_

,iid,est,id,name,brand_id,brand_name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_long,has_variants,has_categories,has_ingredients,has_suggested_usage,is_rated,cologne,men,fragrance,perfume,women,lotions_and_oils,rollerballs_and_travel_size,perfume_gift_sets,value_and_gift_sets,lipstick,lip,makeup,lip_balms_and_treatments,lip_treatments,mascara,eye,lip_gloss,eyebrow,body_sprays_and_deodorant,bath_and_shower,cologne_gift_sets,body_mist_and_hair_mist,concealer,face,lip_stain,deodorant_and_antiperspirant,body_care,bath_and_body,face_primer,foundation,tinted_moisturizer,face_serums,treatments,skincare,mists_and_essences,moisturizers,highlighter,liquid_lipstick,lip_sets,face_sets,setting_spray_and_powder,aftershave,shaving,eye_creams_and_treatments,eye_care,face_masks,masks,face_wash_and_cleansers,cleansers,exfoliators,makeup_removers,eyeliner,face_brushes,makeup_brushes_and_applicators,makeup_palettes,eye_palettes,bb_and_cc_creams,blush,cheek,facial_peels,eyelash_curlers,accessories,eye_sets,mini_size,bronzer,face_oils,contour,lip_liner,eyeshadow,face_wash,moisturizer_and_treatments,toners,body_lotions_and_body_oils,body_moisturizers,scrub_and_exfoliants,self_tanners,face_sunscreen,sun_care,blemish_and_acne_treatments,facial_cleansing_brushes,high_tech_tools,for_face,color_correct,skincare_sets,decollete_and_neck_creams,hand_cream_and_foot_cream,candles_and_home_scents,night_creams,sheet_masks,eye_brushes,beauty_supplements,wellness,facial_rollers,body_sunscreen,cellulite_and_stretch_marks,for_body,body_wash_and_shower_gel,nail,other_needs,shampoo_and_conditioner,hair,hair_styling_and_treatments,conditioner,shampoo,eye_masks,eye_primer,mirrors_and_sharpeners,tools_and_brushes,cheek_palettes,face_wipes,tweezers_and_eyebrow_tools,hair_accessories,hair_tools,gift_cards,false_eyelashes,brush_sets,brush_cleaners,makeup_and_travel_cases,hair_removal,lip_brushes,sponges_and_applicators,makeup_bags_and_travel_cases,hair_masks,curling_irons,hair_straighteners_and_flat_irons,hair_dryers,dry_shampoo,hair_brushes_and_combs,scalp_and_hair_treatments,bath_soaks_and_bubble_bath,hair_styling_products,blotting_papers,hair_primers,hair_oil,teeth_whitening,anti_aging,travel_size,facial_men,candles,diffusers,holistic_wellness,leave_in_conditioner,hair_spray,color_care,best_for,just_arrived,gifts_her,gifts_men,gifts_them,gifts_teens,gift_sets_men,gift_sets,editors_picks,luxury_gifts,sephora_exclusives,perfume_men,grooming_men,hair_men,personal_care_men,just_arrived_men,best_seller_men,vegan,cruelty_free,for_men


In [345]:
def get_reco2(age,
             keywords,
             rel,
             gender,
             budget_max = 1000,
             budget_min = 0,
             n=50):
    
    if budget_min>budget_max:
        return print("Error: min budget is greater than max budget")
    
    else:
    
        products_to_consider = get_products2(keywords, rel, gender) ####

        if len(products_to_consider)== 0 or type(products_to_consider)==str:
            return print("Error: reenter details")
        else:
            uid_query = get_uid(age, "0", "0", "0", "0", "0",
                                "0", "0")

            if uid_query=="error":
                return print("Error: reenter age")
            else:
                products = pd.read_csv("df_sephora_selected.csv")
                topn = get_topn(uid_query, products_to_consider, n=len(products))
                topn = topn.merge(products, how="left", left_on="iid", right_on="id")
                topn = topn[(topn.price_high <= budget_max)
                            & (topn.price_low >= budget_min)].reset_index(
                                drop=True).head(n)
                
    if len(topn) == n:
        return topn
    else:
        ager = age_range(age)
        similar_uids = get_similar_uids(ager)
        
        
        

In [7]:
def show_image(product_id):
    image_folder_path = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/image/"
    display(Image(image_folder_path + product_id + ".png", width=250, height=250))
    
    

In [357]:
def get_similar_uids(ager):
    uids = pickle.load(open("uid.pkl", "rb"))
    similar_uids = [
        uid for uid in uids
        if uid.startswith(ager) and not uid.endswith("_0_0_0_0_0_0_0")
    ]
    similar_uids.sort()

    return similar_uids


def age_range(age):

    age_dict = {
        "13to17": range(1, 17 + 1),
        "18to24": range(18, 24 + 1),
        "25to34": range(25, 34 + 1),
        "35to44": range(35, 44 + 1),
        "45to54": range(45, 54 + 1),
        "over54": range(54, 100 + 1),
        "0": [0]
    }

    if age > 100 or age < 0:
        print("please check age input")
        return "error"

    for key, val in age_dict.items():
        if age in val:
            age_group = key
            if age < 13:
                print("products may not be aged appropriate")
            return age_group


def get_uid(age="0",
            stone="0",
            stype="0",
            sconcerns="0",
            ecolor="0",
            hcolor="0",
            hcondition="0",
            hconcerns="0"):

    ager = age_range(age)
    if ager == "error":
        return "error"

    skintone_options = [
        "0", 'porcelain', 'fair', 'light', 'medium', 'tan', 'olive', 'dark',
        'deep', 'ebony'
    ]

    skintype_options = ["0", "normal", "oily", "dry", "combination"]

    sconcerns_options1 = [
        "0", "acne", "aging", "blackheads", "uneven skintones", "sensitivity",
        "dark circles", "calluses", "pores", "redness", "dullness",
        "cellulite", "sun damage", "cuticles", "stretch marks", "puffiness"
    ]

    sconcerns_options2 = [
        "0", "acne", "aging", "blackheads", "unevenSkinTones", "sensitivity",
        "darkCircles", "calluses", "pores", "redness", "dullness", "cellulite",
        "sunDamage", "cuticles", "stretchMarks", "puffiness"
    ]

    ecolor_options = ["0", 'green', 'blue', 'brown', 'hazel', 'gray']

    hcolor_options = [
        "0", 'brunette', 'black', 'red', 'blonde', 'gray', 'auburn'
    ]

    hcondition_options1 = [
        "0", 'chemically treated', 'normal', 'wavy', 'fine', 'curly', 'coarse',
        'dry', 'oily', 'straight'
    ]

    hcondition_options2 = [
        "0", 'chemicallyTreated', 'normal', 'wavy', 'fine', 'curly', 'coarse',
        'dry', 'oily', 'straight'
    ]

    hconcerns_options1 = [
        "0", 'color protection', 'damage', 'curly enchancing', 'dandruff',
        'volumizing', 'anti aging', 'frizz', 'hold', 'thinning', 'oiliness',
        'straightening & smoothing', 'heat protection', 'shine'
    ]

    hconcerns_options2 = [
        "0", 'ColorProtection', 'Damage', 'CurlyEnhancing', 'Dandruff',
        'Volumizing', 'AntiAging', 'Frizz', 'Hold', 'Thinning', 'Oiliness',
        'StraighteningSmoothing', 'HeatProtection', 'Shine'
    ]

    if stone not in skintone_options:
        return "error"

    if stype not in skintype_options:
        return "error"

    if sconcerns not in sconcerns_options1:
        return "error"
    else:
        idx = sconcerns_options1.index(sconcerns)
        sconcerns = sconcerns_options2[idx]

    if ecolor not in ecolor_options:
        return "error"

    if hcolor not in hcolor_options:
        return "error"

    if hcondition not in hcondition_options1:
        return "error"
    else:
        idx = hcondition_options1.index(hcondition)
        hcondition = hcondition_options2[idx]

    if hconcerns not in hconcerns_options1:
        return "error"
    else:
        idx = hconcerns_options1.index(hconcerns)
        hconcerns = hconcerns_options2[idx]

    uid_query = ager + "_" + stone + "_" + stype + "_" + sconcerns + "_" + ecolor + "_" + hcolor + "_" + hcondition + "_" + hconcerns

    return uid_query


def get_products2(keywords, rel="0", gender="0"):

    relationship = [
        "mom", "girlfriend", "sister", "friend", "dad", "brother", "client",
        "boss", "family", "son", "daughter"
    ]

    if rel not in relationship:
        return "please reenter details (relationship)"

    if "active" in keywords:
        keywords.remove("active")
        keywords.extend(["active day", "sun", "sunscreen", "spf", "sunblock"])

    products_to_consider = []
    products = pd.read_csv("df_sephora_selected.csv")

    keyword_string = "|".join([
        "\s" + key + "\s|\s" + key + "\,|\s" + key + "\." for key in keywords
    ])

    if gender == "he":
        products_to_consider = list(
            products[products.long_desc.str.contains(
                keyword_string, flags=re.IGNORECASE, regex=True, na=False)
                     & (products.for_men == 1)].id.unique())

    elif gender == "she":
        products_to_consider = list(
            products[products.long_desc.str.contains(
                keyword_string, flags=re.IGNORECASE, regex=True, na=False)
                     & (products.for_men == 0)].id.unique())
    
    else:
         products_to_consider = list(
            products[products.long_desc.str.contains(
                keyword_string, flags=re.IGNORECASE, regex=True, na=False)].id.unique())

    return products_to_consider

In [260]:
def get_topn(uid, product_ids, n=50):

    algo = pickle.load(open("SVD_best.pkl", "rb"))

    items_to_pred = [(uid, iid, 0) for iid in product_ids]

    predictions = algo.test(items_to_pred)

    item_predictions = pd.DataFrame({
        "iid": product_ids,
        "est": [pred.est for pred in predictions]
    }).sort_values("est", ascending=False).reset_index(drop=True)

    return item_predictions.head(n)


In [271]:
a = ["active"]

In [29]:
user_matrix = pd.read_csv("userstats.csv")
user_matrix.drop(columns = ["author_id", "review_id"], inplace=True)
user_matrix.drop_duplicates(subset = ["uid"], inplace=True)
user_matrix = user_matrix[user_matrix.uid.isin(author_ids)].reset_index(drop=True)

/Applications/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,2,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
user_matrix2 = user_matrix.drop(columns = ["uid"]).copy()
user_matrix2.replace(["0", 0], np.nan, inplace=True)

In [78]:
user_matrix2["age2"] = user_matrix2.age.replace({
    "13to17": 1,
    "18to24": 2,
    "25to34": 3,
    "35to44": 4,
    "45to54": 5,
    "over54": 6
})

user_matrix2["skintone2"] = user_matrix2.skintone.replace({
    'porcelain': 1,
    'fair': 2,
    'light': 3,
    'medium': 4,
    'tan': 6,
    'olive': 5,
    'dark': 7,
    'deep': 9,
    'ebony': 8
})

user_matrix2["skintype2"] = user_matrix2.skintype.replace({
    "normal": 2,
    "oily": 4,
    "dry": 1,
    "combination": 3
})

user_matrix2["skinconcerns2"] = user_matrix2.skinconcerns.replace(
    dict(
        zip(
            user_matrix2.skinconcerns.value_counts(ascending=True).index,
            range(1,
                  user_matrix2.skinconcerns.nunique() + 1))))

user_matrix2["eyecolor2"] = user_matrix2.eyecolor.replace(
    {'brown':5, 'blue':2, 'green':3, 'hazel':4, 'gray':1})


user_matrix2["haircolor2"] = user_matrix2.haircolor.replace(
    {'brunette':5, 'blonde':1, 'black':6, 'auburn':4, 'red':3, 'gray':2})


user_matrix2["haircondition2"] = user_matrix2.haircondition.replace(
    dict(
        zip(
            user_matrix2.haircondition.value_counts(ascending=True).index,
            range(1,
                  user_matrix2.haircondition.nunique() + 1))))

user_matrix2["hairconcerns2"] = user_matrix2.hairconcerns.replace(
    dict(
        zip(
            user_matrix2.hairconcerns.value_counts(ascending=True).index,
            range(1,
                  user_matrix2.hairconcerns.nunique() + 1))))

In [84]:
user_matrix3 = user_matrix2[[col for col in user_matrix2.columns if col.endswith("2")]].copy()
user_matrix3.columns = [col.strip("2") for col in user_matrix3.columns]
user_matrix3

,age,skintone,skintype,skinconcerns,eyecolor,haircolor,haircondition,hairconcerns
0,nan,nan,nan,nan,nan,nan,nan,nan
1,nan,nan,nan,nan,nan,nan,nan,10.0000
2,nan,nan,nan,nan,nan,nan,nan,12.0000
3,nan,nan,nan,nan,nan,nan,nan,7.0000
4,nan,nan,nan,nan,nan,nan,nan,13.0000
...,...,...,...,...,...,...,...,...
34744,6.0000,6.0000,4.0000,13.0000,5.0000,2.0000,nan,nan
34745,6.0000,6.0000,4.0000,12.0000,5.0000,6.0000,nan,nan
34746,6.0000,6.0000,4.0000,8.0000,5.0000,1.0000,nan,nan
34747,6.0000,6.0000,4.0000,8.0000,5.0000,1.0000,nan,8.0000


In [ ]:
test_uid = "35to44_0_0_0_0_0_0_0"
pd.

In [86]:
user_matrix3.nunique()

age              6 
skintone         9 
skintype         4 
skinconcerns     15
eyecolor         5 
haircolor        6 
haircondition    9 
hairconcerns     13
dtype: int64

In [87]:
user_matrix.to_csv("user_matrix.csv", index=False)
user_matrix2.to_csv("user_matrix2.csv", index=False)
user_matrix3.to_csv("user_matrix3.csv", index=False)

In [ ]:
df_uid_cosine = cosine_similarity(user_matrix3)